# Imports

In [ ]:
import os
from PIL import Image
import shutil
from math import ceil

### Conversion d'images PNG en JPEG dans le dossier courant

In [1]:
def create_jpeg_images():
    """
    Convertit toutes les images PNG du dossier courant en JPEG.
    Les fichiers PNG originaux restent inchangés.
    """

    current_dir = os.getcwd()
    for file_name in os.listdir(current_dir):
        if file_name.lower().endswith('.png'):
            file_path = os.path.join(current_dir, file_name)
            try:
                with Image.open(file_path) as img:
                    rgb_image = img.convert('RGB')
                    new_file_name = os.path.splitext(file_name)[0] + '.jpg'
                    new_file_path = os.path.join(current_dir, new_file_name)
                    rgb_image.save(new_file_path, 'JPEG')
            except Exception as e:
                print(f"Erreur lors de la conversion de {file_name} : {e}")
                
create_jpeg_images()

### Suppression des images PNG inutilisées par la suite

In [ ]:
def delete_png_images():
    """
    Supprime tous les fichiers PNG du dossier courant.
    """
    current_dir = os.getcwd()
    for file_name in os.listdir(current_dir):
        if file_name.lower().endswith('.png'):
            file_path = os.path.join(current_dir, file_name)
            try:
                os.remove(file_path)
            except Exception as e:
                print(f"Erreur lors de la suppression de {file_name} : {e}")
                
delete_png_images()

### Renommage des images du dossier courant

In [ ]:
def rename_files_to_fix_sorting():
    """
    Renomme les fichiers en supprimant le préfixe et en ajoutant des zéros pour le tri numérique
    """
    current_dir = os.getcwd()
    for file_name in os.listdir(current_dir):
        if file_name.endswith('.jpg') or file_name.endswith('.json'):
            base_name, ext = os.path.splitext(file_name)
            try:
                parts = base_name.split('_')
                if len(parts) > 1 and parts[-1].isdigit():
                    number = int(parts[-1])
                    new_base_name = f"{number:04d}"
                    new_file_name = new_base_name + ext
                    old_path = os.path.join(current_dir, file_name)
                    new_path = os.path.join(current_dir, new_file_name)
                    os.rename(old_path, new_path)
            except ValueError:
                print(f"Nom ignoré : {file_name} (pas de numéro valide à la fin)")

rename_files_to_fix_sorting()

### Compression des images en 480x848

In [9]:
def compress_images(video_dir, num_images):
    """
    Compresse les images .jpg dans un dossier donné à une résolution de 848x480.
    
    Args:
        video_dir (str): Chemin du dossier contenant les images .jpg.
        num_images (int): Nombre d'images à modifier.
    """
    image_files = [
        f for f in os.listdir(video_dir)
        if os.path.splitext(f)[-1].lower() == ".jpg"
    ]
    if num_images > len(image_files):
        raise ValueError(f"Le dossier ne contient que {len(image_files)} images .jpg, "
                         f"mais vous avez demandé d'en traiter {num_images}.")

    for i, image_file in enumerate(image_files[:num_images]):
        image_path = os.path.join(video_dir, image_file)
        
        try:
            with Image.open(image_path) as img:
                img_resized = img.resize((848, 480), Image.Resampling.LANCZOS)
                img_resized.save(image_path, quality=85, optimize=True)
        except Exception as e:
            print(f"Erreur lors de la compression de l'image {image_file}: {e}")
            
compress_images(os.getcwd(),1440)

### Séparation des fichiers en dossier de taille donnée

In [ ]:
def split_files_into_parts(input_folder, output_folder, chunk_size=100):
    """
    Divise les fichiers d'un dossier en plusieurs sous-dossiers contenant un maximum de `chunk_size` fichiers.
    
    Args:
        input_folder (str): Chemin du dossier contenant les fichiers originaux.
        output_folder (str): Chemin où les sous-dossiers seront créés.
        chunk_size (int): Nombre maximum de fichiers par sous-dossier.
    """
    os.makedirs(output_folder, exist_ok=True)
    files = sorted([f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.json'))])
    total_parts = ceil(len(files) / chunk_size)
    
    for part in range(total_parts):
        part_folder = os.path.join(output_folder, f"part{part:02}")
        os.makedirs(part_folder, exist_ok=True)

        start_idx = part * chunk_size
        end_idx = start_idx + chunk_size
        chunk_files = files[start_idx:end_idx]

        for file_name in chunk_files:
            src_path = os.path.join(input_folder, file_name)
            dest_path = os.path.join(part_folder, file_name)
            shutil.move(src_path, dest_path)


input_folder = os.getcwd()
output_folder = os.getcwd()
split_files_into_parts(input_folder, output_folder)